This is an exploratory notes page for the final projects. It will be split into three parts, for each dataset we are presented.

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

## Part I: Customer Churn

We will first look at the customer churn dataset, `./data/Customer_Churn_Dataset.csv`.

In [62]:
customer_churn_fn = './data/Customer_Churn_Dataset.csv'
cc_raw = pd.read_csv(customer_churn_fn)
cc_raw.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [45]:
cc_raw.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Simply looking at the features and data types, there's a lot we need to do to this data. For instance, let's try to further examine these mysterious `object` type features.

In [46]:
print(type(cc_raw['PaymentMethod'][3]))

<class 'str'>


Since each of these are strings, we should create a function that takes a `pandas.Series` of strings and appends a one-hot-encoded dataframe of that feature to a new dataframe. Then, we'll loop through each of the features and create a new `pandas.DataFrame` object to contain the one-hot encoded features.

In [63]:
# data cleaning

cc_raw = cc_raw.drop('customerID', axis = 1) # drop customer IDs feature
cc_raw['TotalCharges'] = pd.to_numeric(cc_raw['TotalCharges'], errors = 'coerce') # make string float numeric

# this one-hot encoding solution is inspired by Cybernetic's solution here:
# https://stackoverflow.com/questions/37292872/how-can-i-one-hot-encode-in-python

cc = pd.DataFrame()
exempt_features = ['tenure', 'Churn', 'MonthlyCharges', 'TotalCharges']

def encode(feature, label):
    global cc
    new_feats = pd.get_dummies(feature, prefix = label, drop_first = True)
    cc = pd.concat([cc, new_feats], axis = 1)

# one-hot encode categorical features    
for i in range(len(cc_raw.columns)):
    col_name = cc_raw.columns[i]
    if col_name not in exempt_features:
        encode(cc_raw[col_name], col_name)

cc = pd.concat([cc, cc_raw[['tenure', 'MonthlyCharges', 'TotalCharges']]], axis = 1)
encode(cc_raw['Churn'], 'Churn')
cc = cc.dropna()

cc.head()

,gender_Male,SeniorCitizen_1,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No internet service,...,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure,MonthlyCharges,TotalCharges,Churn_Yes
0,0,0,1,0,0,1,0,0,0,0,...,0,0,1,0,1,0,1,29.85,29.85,0
1,1,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,1,34,56.95,1889.50,0
2,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,1,2,53.85,108.15,1
3,1,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,45,42.30,1840.75,0
4,0,0,0,0,1,0,0,1,0,0,...,0,0,1,0,1,0,2,70.70,151.65,1


In [64]:
X = cc.drop('Churn_Yes', axis = 1)
X = sm.add_constant(X)
y = cc['Churn_Yes']

lm = sm.OLS(y, X).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Churn_Yes   R-squared:                       0.284
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     120.9
Date:                Wed, 20 Nov 2019   Prob (F-statistic):               0.00
Time:                        16:35:44   Log-Likelihood:                -3057.9
No. Observations:                7032   AIC:                             6164.
Df Residuals:                    7008   BIC:                             6328.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
const                                     0.2737      0.103      2.658      0.008       0.072       0.476
gender_Male                              -0.0034      0.009     -0.375      0.707      -0.021       0.014
SeniorCitizen_1                           0.0445      0.013      3.419      0.001       0.019       0.070
Partner_Yes                              -0.0009      0.011     -0.079      0.937      -0.022       0.020
Dependents_Yes                           -0.0202      0.011     -1.766      0.078      -0.043       0.002
PhoneService_Yes                          0.1341      0.095      1.408      0.159      -0.053       0.321
MultipleLines_No phone service            0.1396      0.014      9.679      0.000       0.111       0.168
MultipleLines_Yes                         0.0587      0.024      2.403      0.016       0.011       0.107
InternetService_Fiber optic               0.2104      0.110      1.920      0.055      -0.004       0.425
InternetService_No                       -0.0256      0.016     -1.621      0.105      -0.057       0.005
OnlineSecurity_No internet service       -0.0256      0.016     -1.621      0.105      -0.057       0.005
OnlineSecurity_Yes                       -0.0425      0.025     -1.710      0.087      -0.091       0.006
OnlineBackup_No internet service         -0.0256      0.016     -1.621      0.105      -0.057       0.005
OnlineBackup_Yes                         -0.0113      0.024     -0.462      0.644      -0.059       0.037
DeviceProtection_No internet service     -0.0256      0.016     -1.621      0.105      -0.057       0.005
DeviceProtection_Yes                      0.0046      0.025      0.185      0.853      -0.044       0.053
TechSupport_No internet service          -0.0256      0.016     -1.621      0.105      -0.057       0.005
TechSupport_Yes                          -0.0439      0.025     -1.754      0.079      -0.093       0.005
StreamingTV_No internet service          -0.0256      0.016     -1.621      0.105      -0.057       0.005
StreamingTV_Yes                           0.0638      0.045      1.416      0.157      -0.025       0.152
StreamingMovies_No internet service      -0.0256      0.016     -1.621      0.105      -0.057       0.005
StreamingMovies_Yes                       0.0658      0.045      1.460      0.144      -0.023       0.154
Contract_One year                        -0.1056      0.014     -7.549      0.000      -0.133      -0.078
Contract_Two year                        -0.0700      0.017     -4.110      0.000      -0.103      -0.037
PaperlessBilling_Yes                      0.0449      0.010      4.495      0.000       0.025       0.064
PaymentMethod_Credit card (automatic)    -0.0061      0.014     -0.448      0.654      -0.033       0.020
PaymentMe